---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **3. `도구 호출 에이전트`** *(Tool Calling Agent)* - ➁

---

* **`환경설정`**

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()                           # True

In [ ]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```bash
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

In [ ]:
# ========================================
# 경고 메시지 무시
# ========================================
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
import sys
from pathlib import Path

# 루트 디렉토리를 Python 경로에 추가
root_dir = Path().absolute().parent
sys.path.append(str(root_dir))

print(f"✅ 루트 디렉토리 추가: {root_dir}")

<small>

- 응답 시간: `0.0s`
- ✅ 루트 디렉토리 추가: `루트/20250727-langchain-note`

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 확인
if not os.getenv("GOOGLE_API_KEY"):        
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")    

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
    #model="gemini-3-flash-preview",
    model="gemini-2.5-flash",            
    temperature=1,                          # gemini-3-flash의 경우 temperature = 1으로 설정 권장         
    )

result=gemini_lc.invoke("대한민국의 수도는?")

# 테스트
print(result.content)
print(result.text)

<small>

* `gemini-3-flash-preview`셀 출력: `6.4s`

  ```bash
  [{'type': 'text', 'text': '대한민국의 수도는 **서울**입니다.', 'extras': {'signature': 'EqcECqQEAXLI2nyXElxN4h...생략...'}}]
  ```

  ```markdown
  대한민국의 수도는 **서울**입니다.
  ```

* `gemini-2.5-flash` 셀 출력 (`1.2s`)

  ```markdown
  대한민국의 수도는 **서울**입니다.
  대한민국의 수도는 **서울**입니다.
  ```

---

##### **6) 중간 단계 출력을 사용자 정의 함수로 출력**

* 다음 3개의 함수를 정의 → 중간 단계 출력을 사용자 정의하기

  * **`tool_callback`**: 도구 호출 출력을 처리하는 함수

  * **`observation_callback`**: 관찰 (`Observation`) 출력을 처리하는 함수

  * **`result_callback`**: 최종 답변 출력을 처리하는 함수

In [ ]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from GoogleNews import GoogleNews 
from langchain_experimental.utilities import PythonREPL


# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    
    # 1. 검색 실행
    # 메서드 이름 변경하기: search_by_keyword -> search
    news_tool.search(query)


    # 2. 결과 반환 방식 변경하기: (search_by_keyword() -> result())
    # ERROR: return news_tool.search_by_keyword()[:5]
    return news_tool.result()[:5]

In [ ]:
# 도구 생성
@tool
def python_repl_tool(
    code:Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

print(f"➀ 도구 이름: {search_news.name}")
print(f"➀ 도구 설명: {search_news.description}")
print(f"② 도구 이름: {python_repl_tool.name}")
print(f"② 도구 이름: {python_repl_tool.description}")

<small>

* 셀 출력

```markdown
    ➀ 도구 이름: search_news
    ➀ 도구 설명: Search Google News by input keyword
    ② 도구 이름: python_repl_tool
    ② 도구 이름: Use this to execute python code. If you want to see the output of a value, you should print it out with `print(...)`. This is visible to the user.
```

In [ ]:
# tool 정의

tools = [search_news, python_repl_tool]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent

# LLM 정의
llm = gemini_lc

# 시스템 메시지 내용만 문자열로 정의
system_instruction = (    
                    "You are a helpful assistant. "
                    "Make sure to use the `search_news` tool for searching keyword related news."
                    )

# Agent 생성 (v1.0.5 기준)
agent = create_agent(    
    model=llm,     
    tools=tools,    
    system_prompt=system_instruction,    
    ) 

In [ ]:
from langchain_core.messages import HumanMessage

# Agent 바로 실행하기
# 스트리밍 모드
result = agent.stream(
    {"messages": [HumanMessage(content="AI 투자와 관련된 뉴스를 검색해주세요.")]},
    config={"max_iterations": 10}
)

for step in result:
    # 중간 단계 출력 
    print(step)

<small>

* `gemini-2.5-flash` 셀 출력 (`4.1s`)

  ```bash
  {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "AI \\ud22c\\uc790"}'}, '__gemini_function_call_thought_signatures__': {'ff6cd552-d750-4055-bcf7-45f90b5c961a': 'CsoBAXLI2nwS+QHavQ9DdMlwOWTZmTG6STv1pxfGJuwx6wgZafv5pqxya35gZcsP1cHg16Fc4LoQ+JtCKrjUe3Sd1uH2zuGWw0yivE0mBvcjhdVweCLo4udq8ZGdW2UOHXNfdfYivFxN4XPjxErBgBSWhdAhHdxZ+6m7trCy22H46VJeLlkdN6ofQ9om5UV+spNKC9uDd/HA15ZZcc72j26YBUjrorb31x4TMXZDVSoaLXkFUJEgOr9zurdJAaOaQoLa6Fm+wE0gBdTHyw=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b88b0-7c83-79a2-80e3-bc156868ca93-0', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI 투자'}, 'id': 'ff6cd552-d750-4055-bcf7-45f90b5c961a', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 149, 'output_tokens': 56, 'total_tokens': 205, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 40}})]}}
  ```

<small>

* `gemini-3-flash-preview` 셀 출력 (`12.0s`)

  ```bash
  {'model': {'messages': [AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "AI \\ud22c\\uc790 \\ucd5c\\uc2e0 \\ub274\\uc2a4"}'}, '__gemini_function_call_thought_signatures__': {'11027ec2-73f6-4ccf-b9eb-ec96e98b0518': 'EskCCsYCAXLI2nzeGweNApS46GQyIUJgTLh8pmwGE0GRqVnS0eEPQTdY5nBRZ0W0YKUEa/jrSkv2gERHInJc4pfc5/HkpHJy2+K920f1z8IYl4YmhqZ709PMoa7TEjIGrIWqteUlmWrosrZMObcAPrjv49Vm5YWj77Ch6dlgRdrHeuSMXx/2kjWHb2hfxdKBRFkrsqUH67pGtpyuBq9broS+Na6mNUSOrZyQEyHxEOY576xtslriA3eYpW4jxyHQSAkXNU1Yo2BLwO2vLfAreL5xU3DAg2Lbg0dgzGpA3lHEroq/6IfHxACSELBvubr79m09SeLjPW6EetJh1CvJ4f9c82ep0xSBrnof2VD1ktiWKW3RE04jjytRYMUVsMStX75eknOaQ9RfvNXr/+0qZKpO5VSzF0DP6Ps8CoKTX/FVA6cB2kVPXO67xRQ='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b887e-45da-72c3-bd6f-808a2aa32751-0', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI 투자 최신 뉴스'}, 'id': '11027ec2-73f6-4ccf-b9eb-ec96e98b0518', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 165, 'output_tokens': 87, 'total_tokens': 252, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 67}})]}}
  {'tools': {'messages': [ToolMessage(content='[{\'title\': "Qiming Venture Partners\' Insilico Medicine Successfully Lists on Hong Kong Stock Exchange", \'media\': \'IT비즈뉴스\', \'date\': \'4 days ago\', \'datetime\': datetime.datetime(2025, 12, 31, 19, 12, 17, 765916), \'desc\': \'SHANGHAI, Dec. 31, 2025 /PRNewswire/ -- Insilico Medicine (03696.HK), a global leader in AI-driven drug discovery and a portfolio company of Qiming Venture...\', \'link\': \'https://www.itbiznews.com/news/articleView.html%3Fidxno%3D200612&ved=2ahUKEwjwsvfq0vGRAxXga_UHHWtEHEUQxfQBegQIAxAC&usg=AOvVaw1urhXKFh4PKWpHNMoQ4r9U\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': "China\'s Sleep Technology Pioneer Sleemon Draws Increasing Interest from the Global AI Community", \'media\': \'IT비즈뉴스\', \'date\': \'4 days ago\', \'datetime\': datetime.datetime(2025, 12, 31, 19, 12, 17, 773923), \'desc\': \'SHAOXING, China, Dec. 30, 2025 /PRNewswire/ -- A long-established household brand in China.\', \'link\': \'https://www.itbiznews.com/news/articleView.html%3Fidxno%3D200590&ved=2ahUKEwjwsvfq0vGRAxXga_UHHWtEHEUQxfQBegQIARAC&usg=AOvVaw2CntuKH0pSa9DHYTRrU5Q8\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'FPT Exports Its First Commercial Chip Shipment to the Japanese Market\', \'media\': \'IT비즈뉴스\', \'date\': \'4 days ago\', \'datetime\': datetime.datetime(2025, 12, 31, 19, 12, 17, 781142), \'desc\': \'HANOI, Vietnam, Dec. 30, 2025 /PRNewswire/ -- FPT Corporation (hereafter "FPT") officially.\', \'link\': \'https://www.itbiznews.com/news/articleView.html%3Fidxno%3D200588&ved=2ahUKEwjwsvfq0vGRAxXga_UHHWtEHEUQxfQBegQIChAC&usg=AOvVaw2cL7B0DOaT8vjYK_rYh6Ad\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': "Nota AI to Supply AI Optimization Technology for Samsung Electronics\' Next-Generation Mobile AP \'Exynos 2600\'… Solidifying Its Position as a Leader in On-Device AI", \'media\': \'IT비즈뉴스\', \'date\': \'4 days ago\', \'datetime\': datetime.datetime(2025, 12, 31, 19, 12, 17, 787946), \'desc\': \'Secured consecutive contracts from Exynos 2500 to the next-generation Exynos 2600, proving the superior optimization capabilities...\', \'link\': \'https://www.itbiznews.com/news/articleView.html%3Fidxno%3D200585&ved=2ahUKEwjwsvfq0vGRAxXga_UHHWtEHEUQxfQBegQIAhAC&usg=AOvVaw1QwWnkwQdP04KW_2E6nrPQ\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'SANY Commences Operations at its First Engineering Machinery Remanufacturing Base in Hainan\', \'media\': \'IT비즈뉴스\', \'date\': \'4 days ago\', \'datetime\': datetime.datetime(2025, 12, 31, 19, 12, 17, 794406), \'desc\': \'DONGFANG, China, Dec. 30, 2025 /PRNewswire/ -- SANY Group officially commenced operations.\', \'link\': \'https://www.itbiznews.com/news/articleView.html%3Fidxno%3D200577&ved=2ahUKEwjwsvfq0vGRAxXga_UHHWtEHEUQxfQBegQIBBAC&usg=AOvVaw2txXXAdPMRzZeokdMEqz63\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]', name='search_news', id='5c03457a-5966-46b4-8fdb-e89512e39379', tool_call_id='11027ec2-73f6-4ccf-b9eb-ec96e98b0518')]}}
  {'model': {'messages': [AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "\\ube45\\ud14c\\ud06c AI \\ud22c\\uc790 \\uc804\\ub9dd 2024 2025"}'}, '__gemini_function_call_thought_signatures__': {'76ae20b6-667e-41c8-b229-1367cc20a40c': 'EsUECsIEAXLI2nwr9c8VPRxV7r3mwaVeDjko+zpV96+dvsTrL16WEIvdltlT/AhCsL4jj65RKF60/X61ga5CixP1sWh4byqrud3/bGNx5DvvCD++U926o55QdFzWKzPYUgFg+usrzxlTsz5PgElDhqF+tI1tVKbnW0Oah5wHZ9vcAktUs/ljdNxhWXur9rc1Ycor8kCbnP/gjbTbeYJPhEq/Wc3gU5BaYfWa/qD3Efh/l/hCoOKHJx0KLGzUS1e7Q/9+2Lng9ZQDlfPqEN4TPzjHRGVF2BUkbmpMAYnwiMy3sYBPLtukSDKeBBWaH/VDNvbIGk1QQ5uDW6Mf1HEzkKURjDvl2GUfBItoCaFP0JXEhaNF+rUrx1HkDGCQKeszzzbl9/X2IciShR63CDMjgnnEAbvXyQKl8ivDQIloAM/rMQovKPn+ZJow8KsT7TbjBVrZbkbyK7JcNIhsl0mYdRtyMA2+F81382GmnlrnvS+tP+9YQIyoyaJhuU8DPjXea+/qgrdaD1geZVc8kWCGVTemrqxYAtrSQGOxOQIjWbmUYjd7P4qd65GXjSjE8A1nbeBawc5+X+l7DtiJdJk15eOxMJEV+X0ZNswp2lN4tOn6L9iXXYj0cPDeYeB6ujUaOJoVczbEiNst0Yn8iCoyYS3wy762BeG5s27MbRuq8LQxw6iDe9ONXmGZfF3Iss8VsFRT/Jy0KnShpUEL69dy7FaZy46WJQw2oEvuqWaJIHd+7G68iC+oZuctbPEycGQKMWobVlBx+gg='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b887e-4f2b-7610-b5ba-d102157da536-0', tool_calls=[{'name': 'search_news', 'args': {'query': '빅테크 AI 투자 전망 2024 2025'}, 'id': '76ae20b6-667e-41c8-b229-1367cc20a40c', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1525, 'output_tokens': 145, 'total_tokens': 1670, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 114}})]}}
  {'tools': {'messages': [ToolMessage(content='[{\'title\': \'2026 Investment Outlook\', \'media\': \'BlackRock\', \'date\': \'1 month ago\', \'datetime\': datetime.datetime(2025, 12, 4, 19, 12, 20, 236460), \'desc\': "AI\'s growth is led by a few firms making massive investments, raising investor questions about whether AI revenues will match this scale of spending.", \'link\': \'https://www.blackrock.com/corporate/insights/blackrock-investment-institute/publications/outlook&ved=2ahUKEwid9pXs0vGRAxXUbPUHHWbtMgEQxfQBegQIBxAC&usg=AOvVaw2YZdxmxjQbMduWLvblg7zs\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'2026 investment management outlook\', \'media\': \'Deloitte\', \'date\': \'1 month ago\', \'datetime\': datetime.datetime(2025, 12, 4, 19, 12, 20, 245983), \'desc\': \'While profit and revenue growth have largely been elusive, 2026 promises to present opportunities for investment management firms that reimagine their...\', \'link\': \'https://www.deloitte.com/us/en/insights/industry/financial-services/financial-services-industry-outlooks/investment-management-industry-outlook.html&ved=2ahUKEwid9pXs0vGRAxXUbPUHHWbtMgEQxfQBegQIAxAC&usg=AOvVaw3R9WvOMm2ZVSTduCCtV3_j\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Should you buy or sell AI stocks?: Jon Erlichman\', \'media\': \'BNN Bloomberg\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 12, 20, 254858), \'desc\': \'The AI boom has fueled technology stocks to all-time highs. That record run-up has sparked a big debate — are we in a bubble or is this a...\', \'link\': \'https://www.bnnbloomberg.ca/ticker-take/2025/10/17/should-you-buy-or-sell-ai-stocks-jon-erlichman/&ved=2ahUKEwid9pXs0vGRAxXUbPUHHWbtMgEQxfQBegQIAhAC&usg=AOvVaw349PgdfwyR9Botk8YNeCkr\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Artificial Intelligence Global Report H1 2025 | AI Investment & Deal Trends | Insights\', \'media\': \'Ropes & Gray LLP\', \'date\': \'4 months ago\', \'datetime\': datetime.datetime(2025, 9, 4, 19, 12, 20, 263219), \'desc\': \'Explore AI dealmaking, fundraising, M&A, and private capital trends in our Artificial Intelligence H1 2025 Global Report—insights for investors, executives,...\', \'link\': \'https://www.ropesgray.com/en/insights/alerts/2025/08/artificial-intelligence-h1-2025-global-report&ved=2ahUKEwid9pXs0vGRAxXUbPUHHWbtMgEQxfQBegQIBBAC&usg=AOvVaw3ERWBb3V8ev7Ub6QaPhZxd\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Big Tech lines up over $300bn in AI spending for 2025\', \'media\': \'The Irish Times\', \'date\': \'0 months ago\', \'datetime\': datetime.datetime(2026, 1, 4, 19, 12, 20, 265106), \'desc\': "Big Tech\'s massive spending on artificial intelligence (AI) is set to continue unchecked in 2025 after Amazon topped its rivals with a planned $100 billion...", \'link\': \'https://www.irishtimes.com/business/2025/02/07/big-tech-lines-up-over-300bn-in-ai-spending-for-2025/&ved=2ahUKEwid9pXs0vGRAxXUbPUHHWbtMgEQxfQBegQIBRAC&usg=AOvVaw0lZBUdzFry3ELEg0gKS2bb\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]', name='search_news', id='a98d6210-416a-450e-921e-8b85f6b4943b', tool_call_id='76ae20b6-667e-41c8-b229-1367cc20a40c')]}}
  {'model': {'messages': [AIMessage(content=[{'type': 'text', 'text': 'AI 투자와 관련된 최신 주요 뉴스와 흐름을 정리해 드립니다. 현재 AI 투자는 **\'빅테크의 천문학적 인프라 투자\'**와 **\'수익성에 대한 의문\'**이라는 두 가지 핵심 키워드로 요약할 수 있습니다.\n\n### 1. 글로벌 빅테크의 대규모 투자 가속화\n*   **3,000억 달러(약 400조 원) 규모의 투자:** 아마존, 마이크로소프트, 구글, 메타 등 글로벌 빅테크 기업들이 2025년 한 해 동안 AI 인프라 확충에 3,000억 달러 이상을 쏟아부을 것으로 전망됩니다.\n*   **아마존의 공격적 행보:** 특히 아마존은 단독으로 약 1,000억 달러(약 130조 원) 규모의 투자를 계획하며 라이벌들을 압도하는 데이터 센터 및 AI 칩 투자를 이어가고 있습니다.\n\n### 2. 반도체 및 온디바이스(On-Device) AI 투자\n*   **삼성전자와 노타(Nota) AI:** 국내 AI 최적화 기술 기업인 \'노타 AI\'가 삼성전자의 차세대 모바일 프로세서(AP)인 **\'엑시노스 2600\'**에 최적화 기술을 공급하기로 했습니다. 이는 스마트폰 등 기기 자체에서 AI를 구동하는 \'온디바이스 AI\' 시장의 투자가 구체화되고 있음을 보여줍니다.\n*   **차세대 반도체 수출:** 베트남의 FPT가 일본 시장에 상용 칩 출하를 시작하는 등 글로벌 반도체 공급망 전반에 AI를 겨냥한 투자가 활발합니다.\n\n### 3. 시장의 시각: "거품인가, 성장의 초기인가?"\n*   **수익성 검증 단계:** 블랙록(BlackRock) 등 주요 투자 기관들은 빅테크의 막대한 투자가 실제 매출과 이익으로 연결될 수 있을지에 대해 주목하고 있습니다. 2025~2026년은 AI 서비스가 실제 수익을 창출하는 \'실적 확인의 해\'가 될 것으로 보입니다.\n*   **투자 전략 변화:** 단순한 하드웨어(GPU 등) 투자를 넘어, 이제는 AI를 활용한 신약 개발(인실리코 메디슨의 홍콩 증시 상장 등)이나 스마트 가전 같은 **응용 분야(Application)**로 투자가 다변화되고 있습니다.\n\n### 4. 국내외 주요 뉴스 요약\n| 구분 | 주요 내용 |\n| :--- | :--- |\n| **빅테크** | 아마존·MS 등 2025년 AI 인프라에 3,000억 달러 이상 투자 예정 |\n| **국내 기술** | 노타 AI, 삼성 엑시노스 2600에 AI 최적화 기술 공급 (온디바이스 AI 강화) |\n| **바이오/AI** | AI 신약 개발사 \'인실리코 메디슨\' 홍콩 증시 성공적 상장 |\n| **반도체** | 베트남 FPT, 일본에 첫 상용 칩 수출하며 AI 반도체 시장 가세 |\n\n**요약하자면:** 현재 AI 투자는 **\'인프라 구축(하드웨어)\'**에서 **\'효율화 및 실질 서비스(소프트웨어/최적화)\'**로 중심축이 이동하고 있으며, 투자자들은 거대 자본이 투입된 AI가 실제 수익을 가져다줄 \'킬러 앱\'의 등장을 기다리고 있는 상황입니다.', 'extras': {'signature': 'EvIGCu8GAXLI2nxDVsUeWXGGSPaj4b8ZAwq+jh+L8OZ/PxnM3ruMoitR8FO5bQe4a0w7lgGe4xLL7mEwFijNyLh6k/tVn37dcwcctgaGNvhU0mFR47l3JxfD9qiSSZCjJYMuYZ2xAJWt5bqolC1/DrYhTrj83MpEI2xPkqGSqlT99Mt7kAxfc8HEwt0gCww3hvSrNRojV8opH6M3f0W+c9y8HvxcclxhTlIjVsLuVa+0RAq0ybs/GfMQN6CydkxCX66Kx0Gb8a2aGTGowlRjjFWjRNSlaPkvOuZV9vkHJDt4JZv14WzSPYnFKeLQ4Es6tnjFDmqMRblfdXcIRSLhc/8k6iIhv4xY/w2pk3h6y0oa2x5baMMeSGbx2MDl3HtUgb368AGR07cDT4Lv6pbE7MyEQJXXYGFQ9ViSD0jlgW6yXrhM9X8X34OqUwBiDXka4++AXNM68GM+skc+Nxlt5DNfdo1aSu+Hby3stNz5YORCCyjHIjXrw6FofcdrWQ9sA7H5fngyLW5LJvSQVpBj23g2VV2AFgkbcLfBf82Wm45DBTcqHoZoeLhhci6wElvv/40uCQYH6RzHMrV9fFB85QJJkHgjffsYmnDpmvU4KtCBGMjBoLNGtTR2X9fdZoFDghGL0OYXRtKHPBWD+AzLH53dVGcCxbdPgL8acTKwIkOxTRZJ4h44m0pUeDltttjxYjXH+LbgqM7Ygs+kTxheccj5J4ZXTaU1tJqHQCkwvUv9wHb+olj+46ZnzCS0WvgWNawcuIV0eUPISt2VMdKJHdr52JvwXso+FhWvkja4u13RpX1EjKy30RRJIwiPk9L8qLAVdDZq58xsQ0hwhpzvieI0Snky5WmI4jpc1hwvcitFaG2/wwnJFP9w7WeSkG+PTnlSGhzxIEvAARON7AnCptg04igyYQIcCFemWoMmu9IEmTNg8qGKt1kW/KvAofsKglO5N3drcWcscuqFcrM4jfPR+B0aqeYpLQzBM3IE9CNksFQD7/NdfOQguenGz0kfLgq1jnV9QqnQXcAStHutk7BccPkUc7hW+gnKpu7YWz/UeTmPbKFPb2LjIeNR4wAOqmv78RuQJnyvUuXMrGCVjnbJMu/HpeyqegTC7oH3jSor6QMbg4q+UpBQlbpwZ/Ols18jR3BPBwc8GP/hv9nUqV5rThk4'}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b887e-58b0-7930-a2cb-8efb4b033f1b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2924, 'output_tokens': 1018, 'total_tokens': 3942, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 209}})]}}
  ```

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def parse_agent_stream(stream):
    """
    LangGraph Agent의 스트림 출력을 파싱하여 깔끔하게 보여주는 함수
    """
    for chunk in stream:
        # 1. 모델(LLM)의 응답 단계인 경우
        if "model" in chunk:
            model_messages = chunk["model"]["messages"]
            for message in model_messages:
                if isinstance(message, AIMessage):
                    # 도구 호출이 포함된 경우 (아직 최종 답변 아님)
                    if message.tool_calls:
                        tool_name = message.tool_calls[0]['name']
                        tool_args = message.tool_calls[0]['args']
                        print(f"🛠️ 도구 호출: {tool_name} (인자: {tool_args})")
                    # 최종 답변인 경우 (content가 있는 경우)
                    elif message.content:
                        print(f"🤖 AI 답변: {message.content}")

        # 2. 도구(Tool) 실행 결과 단계인 경우
        elif "tools" in chunk:
            tool_messages = chunk["tools"]["messages"]
            for message in tool_messages:
                if isinstance(message, ToolMessage):
                    print(f"✅ 도구 실행 완료 (결과 길이: {len(message.content)}자)")

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent.stream(
    {"messages" : [HumanMessage(content="matplotlib 을 사용하여 pie 차트를 그리는 코드를 작성하고 실행하세요.")]},
    config={"recursion_limit": 10}
)

# 사용자 정의 파서 함수로 출력하기
# 함수에 스트림 객체를 통째로 전달
# 함수 안에서 for 루프를 돌면서 하나씩 꺼내 처리함
parse_agent_stream(result)

<small>

* `gemini-2.5-flash` 셀 출력 (`4.4s`)

    ```bash
    🛠️ 도구 호출: python_repl_tool (인자: {'code': "\nimport matplotlib.pyplot as plt\n\n# Data for the pie chart\nlabels = ['A', 'B', 'C', 'D']\nsizes = [15, 30, 45, 10]\ncolors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']\nexplode = (0.1, 0, 0, 0)  # explode 1st slice\n\n# Create pie chart\nplt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)\n\n# Equal aspect ratio ensures that pie is drawn as a circle.\nplt.axis('equal')\n\nplt.title('Sample Pie Chart')\nplt.show()\n"})
    ```

    * ![matplotlib_output3](../16_Agent/assets/03_Agent_output3.png)

    * ✅ 도구 실행 완료 (결과 길이: 0자)
    * 🤖 AI 답변: 알겠습니다. matplotlib을 사용하여 파이 차트를 그리는 코드를 작성하고 실행했습니다. 시각적인 차트가 표시되어야 합니다.

<small>

* 셀 출력 (`9.3s`)

  * WARNING:langchain_experimental.utilities.python:Python REPL can execute arbitrary code. Use with caution.
  * 🛠️ 도구 호출: python_repl_tool (인자: {'code': "import matplotlib.pyplot as plt\n\n# 데이터 준비\nlabels = ['사과', '바나나', '딸기', '포도']\nsizes = [25, 30, 25, 20]\ncolors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']\nexplode = (0.1, 0, 0, 0)  # 첫 번째 조각(사과)을 약간 떼어냄\n\n# 차트 그리기\nplt.figure(figsize=(7, 7))\nplt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',\n        shadow=True, startangle=140)\n\n# 한글 폰트 설정 (환경에 따라 깨질 수 있어 기본 폰트 사용 시 주의 필요)\n# 여기서는 표준적인 matplotlib 코드를 제공합니다.\nplt.title('과일 선호도 조사')\nplt.axis('equal')  # 파이 차트가 원형을 유지하도록 설정\n\nplt.show()"})

  * ![matplotlib](../16_Agent/assets/03_Agent_output2.png)

  * ✅ 도구 실행 완료 (결과 길이: 0자)
  * 🤖 AI 답변: [{'type': 'text', 'text': "`matplotlib`을 사용하여 파이 차트(Pie Chart)를 그리는 코드를 작성하고 실행했습니다.\n\n### 파이 차트 생성 코드\n\n```python\nimport matplotlib.pyplot as plt\n\n# 데이터 준비\nlabels = ['Apple', 'Banana', 'Strawberry', 'Grape'] # 항목\nsizes = [25, 30, 25, 20]                           # 비율\ncolors = ['#ff9999','#66b3ff','#99ff99','#ffcc99'] # 색상\nexplode = (0.1, 0, 0, 0)                           # 첫 번째 조각 강조 (돌출)\n\n# 차트 설정\nplt.figure(figsize=(7, 7))\nplt.pie(sizes, \n        explode=explode, \n        labels=labels, \n        colors=colors, \n        autopct='%1.1f%%',   # 비율 표시 형식\n        shadow=True,          # 그림자 효과\n        startangle=140)      # 시작 각도\n\n# 차트 제목 및 비율 설정\nplt.title('Fruit Preference Survey')\nplt.axis('equal')            # 파이 차트가 완벽한 원이 되도록 설정\n\n# 출력\nplt.show()\n```\n\n### 주요 코드 설명:\n1.  **`labels` & `sizes`**: 차트에 표시할 항목의 이름과 각 항목이 차지하는 크기(비율)를 정의합니다.\n2.  **`explode`**: 특정 조각을 원형에서 분리하여 강조하고 싶을 때 사용합니다. 위 코드에서는 첫 번째 항목인 'Apple'을 0.1만큼 밖으로 뺐습니다.\n3.  **`autopct`**: 각 조각 위에 백분율을 자동으로 계산해서 표시해 줍니다 (`%1.1f%%`는 소수점 첫째 자리까지 표시하라는 의미입니다).\n4.  **`startangle`**: 차트의 시작 각도를 정해줍니다. 기본값은 0도(3시 방향)이며, 각도를 조정해 조각들의 배치 방향을 바꿀 수 있습니다.\n5.  **`plt.axis('equal')`**: 원이 타원형으로 일그러지지 않고 정원이 되도록 설정합니다.", 'extras': {'signature': 'EpsCCpgCAXLI2nwJp3c7tbxN537F9oCkC2dFhTWJfHE9GeMegfz/JH6xlCTu823VsAJZYGFY+7P9uMkTsUPaevwgQkR1fy4Zw1W7TqrJRz6S09KFaZNb7UlOfwSzkaDUH+KMuZUimbm38gH9CMAb9Va3fzGY+9H+UCi1Q+h2dLXfged0FrkSmNqTCQ29ktSpojUNWFh51b8KW4bN8uFG9qt39fxyLQd/uos3S6NvQoqrH9lnB1H4rlelvsUp13vVlwRKTt0O3Jo+usVMYVrj0NBHeqHE/gELcBGSy4tVWDJtdsEt0Xo5nWyhBkbAtvVqu6U0aAbtnuJpeE8q8D7cned8HPec79H0DIVBhuuxSZrjLUq+rkXz0gZ9nhp7cg=='}}]

---

* **`callback`** 을 수정해 사용하는 방법

In [ ]:
# 도구 호출 콜백 함수 정의하기

def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    # 사용된 도구의 이름을 출력하기
    print(f"Tool: {tool.get('tool')}")
    print("<<<<<<< 도구 호출 >>>>>>")

In [ ]:
# 관찰 결과를 출력하는 콜백 함수 정의하기

def observation_callback(observation) -> None:
    print("<<<<<<< 관찰 내용 >>>>>>")
    print(
        f"Observation: {observation.get('observation')[0]}"
    )
    
    # 관찰 내용 출력하기
    print("<<<<<<< 관찰 내용 >>>>>>")

In [ ]:
# 최종 결과를 출력하는 콜백 함수 정의하기
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)
    print("<<<<<<< 최종 답변 >>>>>>")

In [ ]:
# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정하기

from langchain_core.callbacks import BaseCallbackHandler

# 1. AgentCallbacks 클래스 직접 정의
class AgentCallbacks(BaseCallbackHandler):
    def __init__(self, tool_callback=None, observation_callback=None, result_callback=None):
        self.tool_callback = tool_callback
        self.observation_callback = observation_callback
        self.result_callback = result_callback
    
    # 도구가 실행될 때 호출 (tool_callback 연결)
    def on_tool_start(self, serialized, input_str, **kwargs):
        if self.tool_callback:
            # 기존 함수가 기대하는 딕셔너리 형태로 변환
            self.tool_callback({'tool': serialized.get('name'), 'tool_input': input_str})
    
    # 도구 실행이 끝났을 때 호출 (observation_callback 연결)
    def on_tool_end(self, output, **kwargs):
        if self.observation_callback:
            self.observation_callback({'observation': [str(output)]})
    
    # 에이전트/체인이 끝났을 때 호출 (result_callback 연결)
    # LangGraph 환경에서는 on_chain_end가 호출
    def on_chain_end(self, outputs, **kwargs):
        if self.result_callback:
            # LangGraph의 결과 = 보통 messages 리스트가 담긴 딕셔너리
            if isinstance(outputs, dict) and "messages" in outputs:
                last_message = outputs["messages"][-1]
                # 마지막 메시지가 AI의 답변이라면 출력
                if hasattr(last_message, "content") and last_message.content:
                    self.result_callback(last_message.content)
            # Legacy AgentExecutor의 경우 output 키에 결과가 담길 수 있음
            elif isinstance(outputs, dict) and "output" in outputs:
                self.result_callback(outputs["output"])

In [ ]:
# 2. 객체 생성하기

agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,
    observation_callback=observation_callback,
    result_callback=result_callback,
)

In [ ]:
# 1. 에이전트 실행 (이때 콜백들이 자동으로 작동하여 로그를 찍습니다)
# parse_agent_stream 함수는 사용하지 않음

result2 = agent.stream(
    {"messages": [HumanMessage
                (content="AI 투자와 관련된 뉴스를 검색해주세요.")]},
    config={
        "callbacks": [agent_callbacks],
        "max_iterations": 10
        }
    )

for step in result2:
    # 중간 단계 출력 
    print(step)

<small>

* `gemini-2.5-flash` 셀 출력 (`5.5s`)

    ```bash
    {'model': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "AI \\ud22c\\uc790"}'}, '__gemini_function_call_thought_signatures__': {'5772c6ee-d682-4230-9fc5-897c0dcd4dd0': 'Cp4CAXLI2nzySHtTN0wwShDKghOIP+1XSDKOK2I1fKXgadbHMGggYUa9VF01/vC8BPI5ME9UOsyTOBo+FWCVXmYx4h7nReKvl0FaaIB0iuWc9GPDS2zwxfid/m9ZkcIIukMqQiw3YJpO1F1CuW3a2Gi/9Qh5oZcbMVBNFBg5ZmOB/pCXECIIQXOKTfa5vwIM3PZfILokgRbhtwPa+s9lriNAU9zHbrlm1y3XxNqeswbDWKXg4Q87dv3FVWqGo3JXdNISI8sgGSzyhMJxG5tS1hpqEytDKgXBZSB4lORtfEFM1Vot7h6jdIus23kt8BgKXSUCdrr76GxqhYmoOqozAEZumDjM6T0zEqwRi6X3iUMbgfQMfxVq3I+dbRjdiU7awA=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b88b4-d64a-7023-9432-4e869bcdb8e2-0', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI 투자'}, 'id': '5772c6ee-d682-4230-9fc5-897c0dcd4dd0', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 149, 'output_tokens': 79, 'total_tokens': 228, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 63}})]}}
    <<<<<<< 도구 호출 >>>>>>
    Tool: search_news
    <<<<<<< 도구 호출 >>>>>>
    <<<<<<< 관찰 내용 >>>>>>
    Observation: content='[{\'title\': \'The request / response that are contrary to the Web firewall security policies have been blocked.\', \'media\': \'전자신문\', \'date\': \'1 minutes ago\', \'datetime\': datetime.datetime(2026, 1, 4, 20, 10, 53, 260797), \'desc\': \'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.\', \'link\': \'https://www.etnews.com/20260103000033&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBhAC&usg=AOvVaw24p8HaMLPtBkoP6fYDZb7p\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.\', \'media\': \'Smartkarma\', \'date\': \'5 days ago\', \'datetime\': datetime.datetime(2025, 12, 30, 20, 11, 53, 280357), \'desc\': \'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...\', \'link\': \'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQICBAC&usg=AOvVaw1nqf3VrJwZs41-Ycrw0E1g\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)\', \'media\': \'www.termokonteiner.ru\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 20, 11, 53, 288093), \'desc\': \'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIChAC&usg=AOvVaw2kF7RFhVldSxhwOvAUqZXY\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta\', \'media\': \'Smartkarma\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 20, 11, 53, 295716), \'desc\': \'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\', \'link\': \'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBxAC&usg=AOvVaw3TemGq9DhvFurtgc6hzcvd\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목\', \'media\': \'Smartkarma\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 20, 11, 53, 303258), \'desc\': \'애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...\', \'link\': \'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIAxAC&usg=AOvVaw37TFkdUTkmu0axiCKUmkjb\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]' name='search_news' tool_call_id='5772c6ee-d682-4230-9fc5-897c0dcd4dd0'
    <<<<<<< 관찰 내용 >>>>>>
    <<<<<<< 최종 답변 >>>>>>
    [{'title': 'The request / response that are contrary to the Web firewall security policies have been blocked.', 'media': '전자신문', 'date': '1 minutes ago', 'datetime': datetime.datetime(2026, 1, 4, 20, 10, 53, 260797), 'desc': 'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.', 'link': 'https://www.etnews.com/20260103000033&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBhAC&usg=AOvVaw24p8HaMLPtBkoP6fYDZb7p', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.', 'media': 'Smartkarma', 'date': '5 days ago', 'datetime': datetime.datetime(2025, 12, 30, 20, 11, 53, 280357), 'desc': 'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...', 'link': 'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQICBAC&usg=AOvVaw1nqf3VrJwZs41-Ycrw0E1g', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)', 'media': 'www.termokonteiner.ru', 'date': '1 week ago', 'datetime': datetime.datetime(2025, 12, 28, 20, 11, 53, 288093), 'desc': 'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.', 'link': 'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIChAC&usg=AOvVaw2kF7RFhVldSxhwOvAUqZXY', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta', 'media': 'Smartkarma', 'date': '1 week ago', 'datetime': datetime.datetime(2025, 12, 28, 20, 11, 53, 295716), 'desc': 'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.', 'link': 'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBxAC&usg=AOvVaw3TemGq9DhvFurtgc6hzcvd', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목', 'media': 'Smartkarma', 'date': '2 weeks ago', 'datetime': datetime.datetime(2025, 12, 21, 20, 11, 53, 303258), 'desc': '애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...', 'link': 'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIAxAC&usg=AOvVaw37TFkdUTkmu0axiCKUmkjb', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]
    <<<<<<< 최종 답변 >>>>>>
    {'tools': {'messages': [ToolMessage(content='[{\'title\': \'The request / response that are contrary to the Web firewall security policies have been blocked.\', \'media\': \'전자신문\', \'date\': \'1 minutes ago\', \'datetime\': datetime.datetime(2026, 1, 4, 20, 10, 53, 260797), \'desc\': \'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.\', \'link\': \'https://www.etnews.com/20260103000033&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBhAC&usg=AOvVaw24p8HaMLPtBkoP6fYDZb7p\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.\', \'media\': \'Smartkarma\', \'date\': \'5 days ago\', \'datetime\': datetime.datetime(2025, 12, 30, 20, 11, 53, 280357), \'desc\': \'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...\', \'link\': \'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQICBAC&usg=AOvVaw1nqf3VrJwZs41-Ycrw0E1g\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)\', \'media\': \'www.termokonteiner.ru\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 20, 11, 53, 288093), \'desc\': \'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIChAC&usg=AOvVaw2kF7RFhVldSxhwOvAUqZXY\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta\', \'media\': \'Smartkarma\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 20, 11, 53, 295716), \'desc\': \'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\', \'link\': \'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIBxAC&usg=AOvVaw3TemGq9DhvFurtgc6hzcvd\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목\', \'media\': \'Smartkarma\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 20, 11, 53, 303258), \'desc\': \'애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...\', \'link\': \'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwi4-PST4PGRAxUoiq8BHV-zGxQQxfQBegQIAxAC&usg=AOvVaw37TFkdUTkmu0axiCKUmkjb\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]', name='search_news', id='e582fb84-23da-4cc5-a70a-c3daf9172024', tool_call_id='5772c6ee-d682-4230-9fc5-897c0dcd4dd0')]}}
    <<<<<<< 최종 답변 >>>>>>
    [{'type': 'text', 'text': 'AI 투자와 관련된 뉴스 검색 결과입니다:\n\n*   **소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수** (Smartkarma, 5일 전): 소프트뱅크가 디지털브리지를 인수하며 AI 분야에 40억 달러를 투자했습니다.\n*   **2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta** (Smartkarma, 1주 전): Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\n*   **TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목** (Smartkarma, 2주 전): TSMC는 애플의 지원을 받아 2nm 공정으로 발전하여 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호하며, 회사의 회복력과 성장 잠재력을 보여줍니다.', 'extras': {'signature': 'CpMLAXLI2nzGWJlL61UW4aO20DiHYbWMKzoqXY+FthIWu4Hi9bPWuYpIV3s90W5WGNotgnvnHUaJc7vVmuo+VhUFqf04ddEWmttvB1GzDXFFL15LRa66H21nxhCHuwtxUe6Ezj9B/tC4z6FV2OX8bziYqJVJLNUmqKixZDVmvVZyFnmG26NUfN1l39yrkA/DniB70HessvFS4ydFO75Dw3e5SVVJobeRrujbzeIvUwTDc7skYn/nfHR2bnOtiEE999V41a4HxiBw8IcsFrxrN2UsZyNxSsb4UEeRf3xNpjzOx0NxJqkvOveyHx1p6CPZAt1LVPYQ2eIaQrVHY+kRRKlSpmaPOeTMFC/jpGyVdZKVqLrKxSFIzGWwGuuFr61pIoe56yN1KVBZPQ7Vpza5MT0cSAKpqb5WX98h/U/6BFqXk6gUluWzS5pF9lwxUD9TTgP4SakdkSit+ewwVD+WaPhZ28zbmnVwqfzcuDtl2pBHG7tfD+WvO6/0KgePmWuZoy/4G0ZwsMm1RGg8h/U76fB+h/w+YX/fh2JR04JQpQoQozsmyb5vuOstGEhw9lL/Y86Och32JHKtLjAs7eO3RnCam5Q/jmCQayFeHhfZ+nod3MUNuDRgAx2KbmWfmAB5AzCqzgs25tgTITpfd4sS5taLW+kSnt765/b67cFuXvx3/nGM0w9oCsYEcDxhdphfPaEwPShdS4ndj3dGPAA9A6DHZqgmw27wY1fAdrRnY/WMYF+8pPzusea/74Ivsb/qTg3S1ZKlrXnppJbEVqLiNqopeZBiz6T4amJPGeGkozhMtnJBCZok/tRoaSKMvU0kT+oja9s7NvJYLVtulrGkf2uufybJwSW+bS4UJM0GnoCKhk+dDZ62tGRvd0ndkIx1CX37Nb7f8jRLnr0CSdSwPe3YnjYeKM9M3cjpXGWXfuikwSEUwQM9/9wa6Pzb9r9CzMdsVIJqhxhvnc7zVUpy4U1TMHNfJxOcwXM69c0o8pzpkb6atsW9ReZ141PqB8ROPPo4oFM4bpYKUI5csUjOJBSXML9Y7TJ7YYkPSkoGFkA+lyDZ9nZs0sGaVwI9BtKSt0OwH64506GO3EQ2RqmRT3FbzdxA1FBVQ/dbGKS01DJyqMMi5Q0ITm+PahpAjR0BbaBceYxlaPfH/livWeyyNnZ6+LCaOpFk+/c+Dket9LhUY5ZcbBNPSXEQN8cLzSon+o6Hk71PMhxTI/vgpDqMlTCGy9VsQbNI8ifd0VFV0FLswvd9sGGJA8yMhIm4xIE7dDj6cpIWABUqnCFq5QQZ02VbLqUXLaFSuJtTh805XHJ7NRS7bcoSGSU5QZdajy917y/O/22IDIJZbf3HALicutBhclnV20YuRON936VyAEpoyw4IF7AZDeIp6vOWuro9/wWtGQuE5m2AXW5YSxh/qACW62IPRyGoMPePmtXeqCoTOwRxty3ahkT6tLA/UtrWm80O/gHeFN4wpqwTgPjNHqUKhYrc6pDMjyYq9tVw0sCuKSu20oWaLwm4uyAvjRAGH7xdVjP5yrtYdhQl+bd2vFTzjBFGxpHm65KvWgLlOqNqJvwTlAyabBwHecbw182n1DVrjt9VRqlF235/phYX+e6+idZwQuywBGNFj6iKjwjL/k+ATqcEikR7doGDjwM5XY/UMaidfHB1KEtsZJRGbAMGx1ra53Y0hm9k58B6Mn0duxbJL7zG6XHyZHI3AOa5E3Ptzx6b251jdTYLFgFv2peAbeit+jDygTsZl9xXv/AILe2US3LjqZXQEoCIpYEIcQxdT0xWoZPoWbx1A+or4odkTipZwWvxddG8ITG3Uipr1Vb+bDMqh8Yfgv3cIibDJxQ0HmtHMIlvmBSBKqaWA2iQxY5aKqFCZ3QV9okEhkbWBngFpiw='}}]
    <<<<<<< 최종 답변 >>>>>>
    {'model': {'messages': [AIMessage(content=[{'type': 'text', 'text': 'AI 투자와 관련된 뉴스 검색 결과입니다:\n\n*   **소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수** (Smartkarma, 5일 전): 소프트뱅크가 디지털브리지를 인수하며 AI 분야에 40억 달러를 투자했습니다.\n*   **2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta** (Smartkarma, 1주 전): Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\n*   **TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목** (Smartkarma, 2주 전): TSMC는 애플의 지원을 받아 2nm 공정으로 발전하여 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호하며, 회사의 회복력과 성장 잠재력을 보여줍니다.', 'extras': {'signature': 'CpMLAXLI2nzGWJlL61UW4aO20DiHYbWMKzoqXY+FthIWu4Hi9bPWuYpIV3s90W5WGNotgnvnHUaJc7vVmuo+VhUFqf04ddEWmttvB1GzDXFFL15LRa66H21nxhCHuwtxUe6Ezj9B/tC4z6FV2OX8bziYqJVJLNUmqKixZDVmvVZyFnmG26NUfN1l39yrkA/DniB70HessvFS4ydFO75Dw3e5SVVJobeRrujbzeIvUwTDc7skYn/nfHR2bnOtiEE999V41a4HxiBw8IcsFrxrN2UsZyNxSsb4UEeRf3xNpjzOx0NxJqkvOveyHx1p6CPZAt1LVPYQ2eIaQrVHY+kRRKlSpmaPOeTMFC/jpGyVdZKVqLrKxSFIzGWwGuuFr61pIoe56yN1KVBZPQ7Vpza5MT0cSAKpqb5WX98h/U/6BFqXk6gUluWzS5pF9lwxUD9TTgP4SakdkSit+ewwVD+WaPhZ28zbmnVwqfzcuDtl2pBHG7tfD+WvO6/0KgePmWuZoy/4G0ZwsMm1RGg8h/U76fB+h/w+YX/fh2JR04JQpQoQozsmyb5vuOstGEhw9lL/Y86Och32JHKtLjAs7eO3RnCam5Q/jmCQayFeHhfZ+nod3MUNuDRgAx2KbmWfmAB5AzCqzgs25tgTITpfd4sS5taLW+kSnt765/b67cFuXvx3/nGM0w9oCsYEcDxhdphfPaEwPShdS4ndj3dGPAA9A6DHZqgmw27wY1fAdrRnY/WMYF+8pPzusea/74Ivsb/qTg3S1ZKlrXnppJbEVqLiNqopeZBiz6T4amJPGeGkozhMtnJBCZok/tRoaSKMvU0kT+oja9s7NvJYLVtulrGkf2uufybJwSW+bS4UJM0GnoCKhk+dDZ62tGRvd0ndkIx1CX37Nb7f8jRLnr0CSdSwPe3YnjYeKM9M3cjpXGWXfuikwSEUwQM9/9wa6Pzb9r9CzMdsVIJqhxhvnc7zVUpy4U1TMHNfJxOcwXM69c0o8pzpkb6atsW9ReZ141PqB8ROPPo4oFM4bpYKUI5csUjOJBSXML9Y7TJ7YYkPSkoGFkA+lyDZ9nZs0sGaVwI9BtKSt0OwH64506GO3EQ2RqmRT3FbzdxA1FBVQ/dbGKS01DJyqMMi5Q0ITm+PahpAjR0BbaBceYxlaPfH/livWeyyNnZ6+LCaOpFk+/c+Dket9LhUY5ZcbBNPSXEQN8cLzSon+o6Hk71PMhxTI/vgpDqMlTCGy9VsQbNI8ifd0VFV0FLswvd9sGGJA8yMhIm4xIE7dDj6cpIWABUqnCFq5QQZ02VbLqUXLaFSuJtTh805XHJ7NRS7bcoSGSU5QZdajy917y/O/22IDIJZbf3HALicutBhclnV20YuRON936VyAEpoyw4IF7AZDeIp6vOWuro9/wWtGQuE5m2AXW5YSxh/qACW62IPRyGoMPePmtXeqCoTOwRxty3ahkT6tLA/UtrWm80O/gHeFN4wpqwTgPjNHqUKhYrc6pDMjyYq9tVw0sCuKSu20oWaLwm4uyAvjRAGH7xdVjP5yrtYdhQl+bd2vFTzjBFGxpHm65KvWgLlOqNqJvwTlAyabBwHecbw182n1DVrjt9VRqlF235/phYX+e6+idZwQuywBGNFj6iKjwjL/k+ATqcEikR7doGDjwM5XY/UMaidfHB1KEtsZJRGbAMGx1ra53Y0hm9k58B6Mn0duxbJL7zG6XHyZHI3AOa5E3Ptzx6b251jdTYLFgFv2peAbeit+jDygTsZl9xXv/AILe2US3LjqZXQEoCIpYEIcQxdT0xWoZPoWbx1A+or4odkTipZwWvxddG8ITG3Uipr1Vb+bDMqh8Yfgv3cIibDJxQ0HmtHMIlvmBSBKqaWA2iQxY5aKqFCZ3QV9okEhkbWBngFpiw='}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b88b4-de02-7f42-a139-38db41589682-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1671, 'output_tokens': 584, 'total_tokens': 2255, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 347}})]}}
    <<<<<<< 최종 답변 >>>>>>
    [{'type': 'text', 'text': 'AI 투자와 관련된 뉴스 검색 결과입니다:\n\n*   **소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수** (Smartkarma, 5일 전): 소프트뱅크가 디지털브리지를 인수하며 AI 분야에 40억 달러를 투자했습니다.\n*   **2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta** (Smartkarma, 1주 전): Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\n*   **TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목** (Smartkarma, 2주 전): TSMC는 애플의 지원을 받아 2nm 공정으로 발전하여 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호하며, 회사의 회복력과 성장 잠재력을 보여줍니다.', 'extras': {'signature': 'CpMLAXLI2nzGWJlL61UW4aO20DiHYbWMKzoqXY+FthIWu4Hi9bPWuYpIV3s90W5WGNotgnvnHUaJc7vVmuo+VhUFqf04ddEWmttvB1GzDXFFL15LRa66H21nxhCHuwtxUe6Ezj9B/tC4z6FV2OX8bziYqJVJLNUmqKixZDVmvVZyFnmG26NUfN1l39yrkA/DniB70HessvFS4ydFO75Dw3e5SVVJobeRrujbzeIvUwTDc7skYn/nfHR2bnOtiEE999V41a4HxiBw8IcsFrxrN2UsZyNxSsb4UEeRf3xNpjzOx0NxJqkvOveyHx1p6CPZAt1LVPYQ2eIaQrVHY+kRRKlSpmaPOeTMFC/jpGyVdZKVqLrKxSFIzGWwGuuFr61pIoe56yN1KVBZPQ7Vpza5MT0cSAKpqb5WX98h/U/6BFqXk6gUluWzS5pF9lwxUD9TTgP4SakdkSit+ewwVD+WaPhZ28zbmnVwqfzcuDtl2pBHG7tfD+WvO6/0KgePmWuZoy/4G0ZwsMm1RGg8h/U76fB+h/w+YX/fh2JR04JQpQoQozsmyb5vuOstGEhw9lL/Y86Och32JHKtLjAs7eO3RnCam5Q/jmCQayFeHhfZ+nod3MUNuDRgAx2KbmWfmAB5AzCqzgs25tgTITpfd4sS5taLW+kSnt765/b67cFuXvx3/nGM0w9oCsYEcDxhdphfPaEwPShdS4ndj3dGPAA9A6DHZqgmw27wY1fAdrRnY/WMYF+8pPzusea/74Ivsb/qTg3S1ZKlrXnppJbEVqLiNqopeZBiz6T4amJPGeGkozhMtnJBCZok/tRoaSKMvU0kT+oja9s7NvJYLVtulrGkf2uufybJwSW+bS4UJM0GnoCKhk+dDZ62tGRvd0ndkIx1CX37Nb7f8jRLnr0CSdSwPe3YnjYeKM9M3cjpXGWXfuikwSEUwQM9/9wa6Pzb9r9CzMdsVIJqhxhvnc7zVUpy4U1TMHNfJxOcwXM69c0o8pzpkb6atsW9ReZ141PqB8ROPPo4oFM4bpYKUI5csUjOJBSXML9Y7TJ7YYkPSkoGFkA+lyDZ9nZs0sGaVwI9BtKSt0OwH64506GO3EQ2RqmRT3FbzdxA1FBVQ/dbGKS01DJyqMMi5Q0ITm+PahpAjR0BbaBceYxlaPfH/livWeyyNnZ6+LCaOpFk+/c+Dket9LhUY5ZcbBNPSXEQN8cLzSon+o6Hk71PMhxTI/vgpDqMlTCGy9VsQbNI8ifd0VFV0FLswvd9sGGJA8yMhIm4xIE7dDj6cpIWABUqnCFq5QQZ02VbLqUXLaFSuJtTh805XHJ7NRS7bcoSGSU5QZdajy917y/O/22IDIJZbf3HALicutBhclnV20YuRON936VyAEpoyw4IF7AZDeIp6vOWuro9/wWtGQuE5m2AXW5YSxh/qACW62IPRyGoMPePmtXeqCoTOwRxty3ahkT6tLA/UtrWm80O/gHeFN4wpqwTgPjNHqUKhYrc6pDMjyYq9tVw0sCuKSu20oWaLwm4uyAvjRAGH7xdVjP5yrtYdhQl+bd2vFTzjBFGxpHm65KvWgLlOqNqJvwTlAyabBwHecbw182n1DVrjt9VRqlF235/phYX+e6+idZwQuywBGNFj6iKjwjL/k+ATqcEikR7doGDjwM5XY/UMaidfHB1KEtsZJRGbAMGx1ra53Y0hm9k58B6Mn0duxbJL7zG6XHyZHI3AOa5E3Ptzx6b251jdTYLFgFv2peAbeit+jDygTsZl9xXv/AILe2US3LjqZXQEoCIpYEIcQxdT0xWoZPoWbx1A+or4odkTipZwWvxddG8ITG3Uipr1Vb+bDMqh8Yfgv3cIibDJxQ0HmtHMIlvmBSBKqaWA2iQxY5aKqFCZ3QV9okEhkbWBngFpiw='}}]
    <<<<<<< 최종 답변 >>>>>>
    ```

<small>

* `gemini-3-flash-preview` 셀 출력 (`14.7s`)

    ```bash
    {'model': {'messages': [AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "AI \\ud22c\\uc790"}'}, '__gemini_function_call_thought_signatures__': {'9dbcc1ca-37ea-42b6-aff2-0fe870688756': 'EqAECp0EAXLI2nwbioTfkjj4wFqaqSCnlyd4w9DrQieV4DCfY58XHYesA+wrM7Vea9MxAojE2s92AzdPbnECWbi0cCTyAw2TeXlMjcRJIGzkDNd3yeWojj/9Rfjx4ckF/bVtI7OoRF/Uoj44HVi/knBxEM0aDwDJt6JkC/6MEy+BkEEnvIQP2GNKstlpybVYNjIPBja687SJPqjdI8uWGtt3L/FAlDlPUl4DFbvlIPrKO2oQOwtFDP/MN9xkXs6fa7RQwCXGplJmoFltLd+I0zsvuNx8yCcohdjPYr5Wg/VAU/3H11WAiTP/AcWkp0ZPL6+GZ1e4BbdIZxOG73zUkeJ/BWQgGXrr92y3eEipRx6JzFETt+ArNooFo6336oKiCYacK/hIjS1fAd1tBMT1j4ZOJB4IMsZ4RUC6rPReRJls9IMNz3lSh2VYkRQ5umHnAbyZcaH3iUYeP70hBcr4ZPwRgQWH7AQJ4Z69FRZE5UzqikFvF+tL4dVpWv13hjvlQbv3Cq7Tugg0sE+c2uUnhM5YxDrp9KqXR/k8GbqfWCux1VBP9Jbe3E9JYSO6aeT28UcLzo9kUTj3oYu9+ogAi8xWJgG9H8w+HjL5h0sV7Dfmi95DW+rnR3s2plLz50P/4RQXhy+MMo40Ql8SAoUar7oqzfAfwXUvCF14jUWvBUOaPcBRaPFMuH0An/43kTNRmOOyQIfwxgA06TwZ6KLCwNuyew=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b8899-9854-7301-ac6c-97a5d3031e36-0', tool_calls=[{'name': 'search_news', 'args': {'query': 'AI 투자'}, 'id': '9dbcc1ca-37ea-42b6-aff2-0fe870688756', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 165, 'output_tokens': 135, 'total_tokens': 300, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 118}})]}}
    ```
    ```markdown
    <<<<<<< 도구 호출 >>>>>>
    Tool: search_news
    <<<<<<< 도구 호출 >>>>>>
    <<<<<<< 관찰 내용 >>>>>>
    Observation: content='[{\'title\': \'The request / response that are contrary to the Web firewall security policies have been blocked.\', \'media\': \'전자신문\', \'date\': \'1 minutes ago\', \'datetime\': datetime.datetime(2026, 1, 4, 19, 41, 8, 375967), \'desc\': \'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.\', \'link\': \'https://www.etnews.com/20260103000033&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQICRAC&usg=AOvVaw1lBmX2thPhUEfWofBd1KbH\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.\', \'media\': \'Smartkarma\', \'date\': \'5 days ago\', \'datetime\': datetime.datetime(2025, 12, 30, 19, 42, 8, 397332), \'desc\': \'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...\', \'link\': \'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIARAC&usg=AOvVaw28VaHrNTUiTh7L-0wwOqgk\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)\', \'media\': \'www.termokonteiner.ru\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 19, 42, 8, 405313), \'desc\': \'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIChAC&usg=AOvVaw3tMCi9mu8IiaCdD57K_Ifm\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta\', \'media\': \'Smartkarma\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 19, 42, 8, 412727), \'desc\': \'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\', \'link\': \'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIAhAC&usg=AOvVaw1k2CvtpjRXaT17uSL43zC7\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목\', \'media\': \'Smartkarma\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 19, 42, 8, 420055), \'desc\': \'애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...\', \'link\': \'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIBxAC&usg=AOvVaw0R8GZqT40lBiJz0KydDW4E\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]' name='search_news' tool_call_id='9dbcc1ca-37ea-42b6-aff2-0fe870688756'
    <<<<<<< 관찰 내용 >>>>>>
    <<<<<<< 최종 답변 >>>>>>
    [{'title': 'The request / response that are contrary to the Web firewall security policies have been blocked.', 'media': '전자신문', 'date': '1 minutes ago', 'datetime': datetime.datetime(2026, 1, 4, 19, 41, 8, 375967), 'desc': 'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.', 'link': 'https://www.etnews.com/20260103000033&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQICRAC&usg=AOvVaw1lBmX2thPhUEfWofBd1KbH', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.', 'media': 'Smartkarma', 'date': '5 days ago', 'datetime': datetime.datetime(2025, 12, 30, 19, 42, 8, 397332), 'desc': 'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...', 'link': 'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIARAC&usg=AOvVaw28VaHrNTUiTh7L-0wwOqgk', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)', 'media': 'www.termokonteiner.ru', 'date': '1 week ago', 'datetime': datetime.datetime(2025, 12, 28, 19, 42, 8, 405313), 'desc': 'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.', 'link': 'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIChAC&usg=AOvVaw3tMCi9mu8IiaCdD57K_Ifm', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': '2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta', 'media': 'Smartkarma', 'date': '1 week ago', 'datetime': datetime.datetime(2025, 12, 28, 19, 42, 8, 412727), 'desc': 'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.', 'link': 'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIAhAC&usg=AOvVaw1k2CvtpjRXaT17uSL43zC7', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목', 'media': 'Smartkarma', 'date': '2 weeks ago', 'datetime': datetime.datetime(2025, 12, 21, 19, 42, 8, 420055), 'desc': '애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...', 'link': 'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIBxAC&usg=AOvVaw0R8GZqT40lBiJz0KydDW4E', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]
    <<<<<<< 최종 답변 >>>>>>
    {'tools': {'messages': [ToolMessage(content='[{\'title\': \'The request / response that are contrary to the Web firewall security policies have been blocked.\', \'media\': \'전자신문\', \'date\': \'1 minutes ago\', \'datetime\': datetime.datetime(2026, 1, 4, 19, 41, 8, 375967), \'desc\': \'Detect time, 2026-01-04 19:30:46. Detect client IP, 66.249.74.167. Detect URL, https://www.etnews.com/20260103000033.\', \'link\': \'https://www.etnews.com/20260103000033&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQICRAC&usg=AOvVaw1lBmX2thPhUEfWofBd1KbH\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'아시아 부동산 동향 (2025년 12월 30일 기준): 소프트뱅크, AI 분야에 40억 달러 투자하며 디지털브리지 인수.\', \'media\': \'Smartkarma\', \'date\': \'5 days ago\', \'datetime\': datetime.datetime(2025, 12, 30, 19, 42, 8, 397332), \'desc\': \'Softbank has invested $4 billion in AI through the acquisition of DigitalBridge, signaling growth. Casino mogul Lawrence Ho sold his Hong Kong residence to...\', \'link\': \'https://www.smartkarma.com/ko/insights/asia-real-estate-tracker-30-dec-2025-softbank-invests-4b-in-ai-acquiring-digitalbridge&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIARAC&usg=AOvVaw28VaHrNTUiTh7L-0wwOqgk\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'개매디 포커 공략 에 대한 나의 경험: 실제 사례와 교훈 (2025년 최신판)\', \'media\': \'www.termokonteiner.ru\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 19, 42, 8, 405313), \'desc\': \'NHN 사옥./사진=뉴스1 [파이낸셜뉴스] 한국투자증권이 "2026년 게임사업부의 반등이 기대된다"며 NHN의 목표주가를 기존 3만5000원에서 4만원으로 상향 조정했다.\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-28/7847.html&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIChAC&usg=AOvVaw3tMCi9mu8IiaCdD57K_Ifm\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'2026년 고확신 투자 아이디어 - AI의 핵심 에너지 확보: 성장 잠재력 측면에서는 Eaton, 방어 측면에서는 Quanta\', \'media\': \'Smartkarma\', \'date\': \'1 week ago\', \'datetime\': datetime.datetime(2025, 12, 28, 19, 42, 8, 412727), \'desc\': \'Eaton과 Quanta Services는 AI 기반 에너지 수요를 활용하여 전력망 확장 및 데이터 센터 전력 사용을 포괄함으로써 다각화된 노출과 회복력을 확보합니다.\', \'link\': \'https://www.smartkarma.com/ko/insights/2026-high-conviction-idea-capturing-ai-s-energy-backbone-eaton-for-upside-quanta-for-defence&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIAhAC&usg=AOvVaw1k2CvtpjRXaT17uSL43zC7\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'TSMC: AI로 인한 경기 침체에도 흔들리지 않는, 탄탄한 장기 투자 종목\', \'media\': \'Smartkarma\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 19, 42, 8, 420055), \'desc\': \'애플의 지원을 받아 TSMC가 2nm 공정으로 발전한 것은 AI 관련 투자 감소로 인한 잠재적 타격으로부터 회사를 보호해 주며, 회사의 회복력과 성장 잠재력을 보여...\', \'link\': \'https://www.smartkarma.com/ko/insights/tsmc-a-resilient-long-holding-insulated-from-an-ai-slowdown&ved=2ahUKEwiw5-DA2fGRAxXXsVYBHUhVKeUQxfQBegQIBxAC&usg=AOvVaw0R8GZqT40lBiJz0KydDW4E\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]', name='search_news', id='2787856c-c407-4cff-b0c1-20fc5e26b583', tool_call_id='9dbcc1ca-37ea-42b6-aff2-0fe870688756')]}}
    {'model': {'messages': [AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "\\ud55c\\uad6d AI \\ud22c\\uc790 \\uc815\\ucc45 \\uae30\\uc5c5"}'}, '__gemini_function_call_thought_signatures__': {'8b035f1a-8214-4d3f-9ad1-0a06f118a764': 'Er8KCrwKAXLI2nzMl+orDLPhLpCnt4Lbcyoz1jj0dV5Jom4ukxIrPznExZzxOSWmgIGfPRE+8OQVt+AhQeGNYw870a2Av6FK+AWmTH77n0JD4xRlHhYh3URlOCyX4AeeVQyZArhXCnfd4ldt5V9QMBIxPCpxd7SVqtkvuP+z7L2cQzQqqgSuhrkenrJmIo17Ra33q17olzoR6JrBKj049RO1GfQoGx3g+wM79P1DJ4XhZKzdt5FXD8B9iT461tUhq6k01a//3sv79P1dU0LTM27Nv2IrAvZJsDt3yObmV3+ksSV3nRWgmnjfRkUnoBnyxwQTyjl5hUaqsgUTd4djSR2e37bGARjX7IwA6/aIC6s2njqmdIOj1wNCGztUkNne0BfWsGLqQDVILmMDNa4WMvuKJbukCfsf4r/lj9jJwXcyiYhuDKkRlOwDlg0WX+1rzoh4NpocXPxaQH1sjvmzjvl9/OxBdLHYGtP1OJxGVBp2UTgIVfzQtHb0rs/yUmQ5aqRCQEof61n/je29lIi8quSYNVnkkdGGaApQvARM8LqmZ10Tel7dyOFmfaepeRS7fioK2oL0GrZv2aHypPtVJZo6VkcOK5fShuRNUJ4dH7S/jWirljbGhojVrvx9PkTIAQbCRaPEH6ujqi+EqkV6cgizsCaNamz6fUYZHpvscP8LOx7OIa5aLx9DPN0fbhIMiTvsrVpZ2Xv/8WHV9Zl9bhhJHFvGbg1x5YlULIXzpU6B9L0weygaNCTbAloc5tC6KeNVlXVUwElLT/g7rXh3vPu7+k+VI4y4YjvRFS8Pomk4jEES4oG59TzrlE8W8h8D5lJQQXCbzWtV9qVaP3+qFuPnly3Mki1k4F2wk1uX/QjwZAk92xqnINtU0j4p9pPokN4xQYd4sRC5BFo6sVwwtU+UCaoePuksHKzyVnRMMbjd+Xce/61bYasShsxA7C6NGYKwfhu4NhM77/0tjGbZPDz/sAQm9eL8BlCgbVZyIDQ5ilXlUHRrUazwgf2ZZ1WV2LmdQoEXrV6pu0jKJ1r2MI4ABlHLTpG+/SU7B1whGEqUiSxGpcUytLore0K/pEoNITsC41LIfOxQNE54CDG1ZX3BCMQGw+oyHDK5U2rHhFSYkwAncqXcAYc9qP0IpEye6AzE7hLiWRaJNqwXz9FI+WnEP+KSRpAz3pEA55j19mNL+WAfdsj1paZUd3NHctEGZoTi00UycCVazGQggVcEX7IY2WXyHC54rE1XdiOhInle9H2eVHucS5sfVDOkeDsrjftKRVl95QI0XbzvVHNPgC0ifWBgPt1MvnyeUl/j8sGEWpPvxARWK80w7qt9Su3f56j8iw1ExS13pRjh97xpH0v/MTnobmZmau2DNi0Y9JrcUdvI2mTUiB9hqHC+WY4I6YqTyJ7L3rC7P8le6WgGQo1hMBRbfAzv0kKSobIRd3F1i6KEIDKdeqapIRC9207DdQvsWsYoq1MyNAByeqSFLfFqq5Z4/veHLhMfO3SEIo2b3/31yxaYd0eejT2a8oKWnX8V1SqAT3WqO3HIq13SsGVNBxUsRbHOZpUWjYazzQ7v0sRIEhQ54yxBSQEHM20wISGdg9jhcdK7f4lN0V1NiLwSavTPt0FGMcN+oQpln+QD9K1YZQYeQm3sGbssGj9rTOu5uOK5yZyQz/gWHcK8r8HBzw0dv+w4Vy2PbTYbm5I4TT2mnQ+0R4kCw78sV/otAfPhMVsfZL0EKidDFZFBk2KCr6hS0YEnQK6FWeKEJI4uv9x/JU0fJC6zQ7dyeZtcwfw='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b8899-a1cf-74f0-93b6-a56bd3abd455-0', tool_calls=[{'name': 'search_news', 'args': {'query': '한국 AI 투자 정책 기업'}, 'id': '8b035f1a-8214-4d3f-9ad1-0a06f118a764', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 1729, 'output_tokens': 282, 'total_tokens': 2011, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 262}})]}}
    <<<<<<< 도구 호출 >>>>>>
    Tool: search_news
    <<<<<<< 도구 호출 >>>>>>
    <<<<<<< 관찰 내용 >>>>>>
    Observation: content="[{'title': 'AI Revolutionizes Drug Development, Halving Timelines', 'media': '히트뉴스', 'date': '3 months ago', 'datetime': datetime.datetime(2025, 10, 4, 19, 42, 11, 617502), 'desc': 'In a noteworthy global trend, South Korean pharmaceutical companies are actively embracing artificial intelligence (AI) for the development of novel drugs.', 'link': 'https://www.hitnews.co.kr/news/articleView.html%3Fidxno%3D51875&ved=2ahUKEwi2la3C2fGRAxU-klYBHbCgNf0QxfQBegQIARAC&usg=AOvVaw0-k3L_9W0ypaWIWew5YyZB', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]" name='search_news' tool_call_id='8b035f1a-8214-4d3f-9ad1-0a06f118a764'
    <<<<<<< 관찰 내용 >>>>>>
    <<<<<<< 최종 답변 >>>>>>
    [{'title': 'AI Revolutionizes Drug Development, Halving Timelines', 'media': '히트뉴스', 'date': '3 months ago', 'datetime': datetime.datetime(2025, 10, 4, 19, 42, 11, 617502), 'desc': 'In a noteworthy global trend, South Korean pharmaceutical companies are actively embracing artificial intelligence (AI) for the development of novel drugs.', 'link': 'https://www.hitnews.co.kr/news/articleView.html%3Fidxno%3D51875&ved=2ahUKEwi2la3C2fGRAxU-klYBHbCgNf0QxfQBegQIARAC&usg=AOvVaw0-k3L_9W0ypaWIWew5YyZB', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]
    <<<<<<< 최종 답변 >>>>>>
    {'tools': {'messages': [ToolMessage(content="[{'title': 'AI Revolutionizes Drug Development, Halving Timelines', 'media': '히트뉴스', 'date': '3 months ago', 'datetime': datetime.datetime(2025, 10, 4, 19, 42, 11, 617502), 'desc': 'In a noteworthy global trend, South Korean pharmaceutical companies are actively embracing artificial intelligence (AI) for the development of novel drugs.', 'link': 'https://www.hitnews.co.kr/news/articleView.html%3Fidxno%3D51875&ved=2ahUKEwi2la3C2fGRAxU-klYBHbCgNf0QxfQBegQIARAC&usg=AOvVaw0-k3L_9W0ypaWIWew5YyZB', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]", name='search_news', id='4db75a92-5bc4-4000-b265-9d069882dd47', tool_call_id='8b035f1a-8214-4d3f-9ad1-0a06f118a764')]}}
    {'model': {'messages': [AIMessage(content=[], additional_kwargs={'function_call': {'name': 'search_news', 'arguments': '{"query": "\\uc815\\ubd80 AI \\ud22c\\uc790 \\uc608\\uc0b0 2025 2026"}'}, '__gemini_function_call_thought_signatures__': {'737427d2-d62f-49cc-85e1-8ad2fecc4661': 'EuIGCt8GAXLI2nwxJQ1vFSBYjIXJtnxiw2bWZ4IHGUBWtKpiGwLs1a9qFrCNkV2GFmhnx9E5hGGnIszGpAVIuyMuaEWGlbak6vthoefckjvutR7gnm+eusm5kZeYOlJz8HdeL/mDSLeXe8bbsGrglV1OnSRMfKhqrvMOE5RTr1kjC50EIiatLlM54Gb9Jq973LPvfEMlkB5E7AyGUN8UWspdlUjireuyASJPg9ezqLEtCQlg6QmwogS2LDgZClh9UpO9vqeCIj7cRdgV170eu+XDdd173ItKqRZ6AlxOZT6UAnKX64G1vHxbSoruGV03gzugJELlYFyZgnGessZWGBP+0uW+oN0jZmVt9QBQgLRL2nEPNctWcXI3cQJFKL1lj3b+vg5/hBbmFeHR8nNyA4bxfT1NOxRoiuy+oE0ajWPG+W/CYizTnQvm1/SEJeOKko7fIM26PYfKpiRRmEpY4j3UA94lctChODlH0RAnmz8829fr0lMt+9b3uIwV6xfI545YWfDWCkmnsoZx3PoqbDZFq5xaZLxspE4Q4v39h1qd8WyPyY8f71jOg3wstJpnQgrg1c6lT9Z0mdtbDaX7EAgWRVYGIuFzujqurggxJOipwy5m/r3LzchsVsTNWp3urL6jPgKtMstXvPhd/J3JkQ5VYU7DX6CgKR8COXpAHmvi9JAzNXGGKU5OouyrHKg+LtmBWCe7bSw48wfZ/vsj+08ourBAsjvH8dmv2CTSwXD/adqIR8IP2vMF1v1CbIOQfTs/hmXQVkvLCmq/AbsuyYvPBrunriTL/cHsrs3BYPNxSW8Zih73MoRp5BNXTG0WJ55eLk9hdWXNWnU8eU2tIG5965YBD0o+PVVQg22SXobE3x9ylIH3zFPSUV7HCJHFoVpx1a2++1YlBfZ0FgUsggEUnJFwwTEL2EwxH1L9DB/jG6JQ0j9H3uan4jwvYaec0rvf3qQtZzudKwB0smwc9ddeAGPlsUa30af64zDbnhK5nson8opN7sDp6SttCNCpyNT5tvGcz9M5N1zjpmV2W17BAQ2sHcsLgJ+uOSmInwCz5EmXOKoTDw+vSvXSOy6qMpuIdJVvQxCw61KnkPktHDJHuRaw9OtzebI+0F+RG02a83JZUoroN2umvDxepMLc05sZkjg='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b8899-ae2b-77c2-ae51-f7775e6e56e7-0', tool_calls=[{'name': 'search_news', 'args': {'query': '정부 AI 투자 예산 2025 2026'}, 'id': '737427d2-d62f-49cc-85e1-8ad2fecc4661', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 2261, 'output_tokens': 221, 'total_tokens': 2482, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 190}})]}}
    <<<<<<< 도구 호출 >>>>>>
    Tool: search_news
    <<<<<<< 도구 호출 >>>>>>
    <<<<<<< 관찰 내용 >>>>>>
    Observation: content='[{\'title\': \'해외 축구 라이브 의 비밀을 풀다: 전문가들이 공유하는 핵심 팁\', \'media\': \'www.termokonteiner.ru\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 19, 42, 14, 585515), \'desc\': \'해외축구 무료실시간tv중계. 추경 예산안 통과에 인사하는 다카이치 총리와 각료들 [AFP=연합뉴스 자료사진 제공] 일본 정부가 고물가 대응,방위력 강화 등을 위해...\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-20/6637.html&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBhAC&usg=AOvVaw1gfXGg1PjyCfIDdh0vrBht\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'President unveils record annual budget, emphasizes AI as national priority\', \'media\': \'The Korea Times\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 594356), \'desc\': "President Lee Jae Myung on Tuesday called on the National Assembly to approve next year\'s budget, unveiling a record 728 trillion won ($525 billion...", \'link\': \'https://www.koreatimes.co.kr/southkorea/politics/20251104/president-unveils-record-annual-budget-emphasizes-ai-as-national-priority&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBBAC&usg=AOvVaw2PwPBTjxBsV_xMNeY_Xu08\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Federal government plans tax changes to enable $1 trillion boost to economy\', \'media\': \'BNN Bloomberg\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 602917), \'desc\': "Prime Minister Mark Carney\'s government announced plans to enable $1 trillion in investments to drive productivity and boost the Canadian economy to enhance...", \'link\': \'https://www.bnnbloomberg.ca/business/2025/11/04/federal-government-plans-tax-changes-to-enable-1-trillion-boost-to-economy/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIAhAC&usg=AOvVaw1JcO6iTEXe2vU6Mj8vRFdL\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': "Key targets in Greece\'s 2026 draft budget", \'media\': \'Reuters\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 611457), \'desc\': \'The Greek government submitted its 2026 draft budget plan to parliament on Monday, in which it forecasts economic growth of 2.4% in 2026 - outperforming...\', \'link\': \'https://www.reuters.com/business/key-targets-greeces-2026-draft-budget-2025-10-06/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBRAC&usg=AOvVaw3LbDIRxJO0x8SFE5a-cYzg\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Korea boosts R&D budget to record 35.3 trillion won for innovation and growth - CHOSUNBIZ\', \'media\': \'Chosunbiz\', \'date\': \'4 months ago\', \'datetime\': datetime.datetime(2025, 9, 4, 19, 42, 14, 619989), \'desc\': \'Minister Bae Kyung-hun of the Ministry of Science and ICT visits the Korea Research Institute of Standards and Science (KRISS) in Yuseong-gu, Daejeon,...\', \'link\': \'https://biz.chosun.com/en/en-science/2025/08/22/H4U3MOUSRNAFVH6EXGR7KFJQCE/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIARAC&usg=AOvVaw2y7lLm9MMFwuiF8hczDFF7\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]' name='search_news' tool_call_id='737427d2-d62f-49cc-85e1-8ad2fecc4661'
    <<<<<<< 관찰 내용 >>>>>>
    <<<<<<< 최종 답변 >>>>>>
    [{'title': '해외 축구 라이브 의 비밀을 풀다: 전문가들이 공유하는 핵심 팁', 'media': 'www.termokonteiner.ru', 'date': '2 weeks ago', 'datetime': datetime.datetime(2025, 12, 21, 19, 42, 14, 585515), 'desc': '해외축구 무료실시간tv중계. 추경 예산안 통과에 인사하는 다카이치 총리와 각료들 [AFP=연합뉴스 자료사진 제공] 일본 정부가 고물가 대응,방위력 강화 등을 위해...', 'link': 'https://www.termokonteiner.ru/%3FGameID%3D2025-12-20/6637.html&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBhAC&usg=AOvVaw1gfXGg1PjyCfIDdh0vrBht', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'President unveils record annual budget, emphasizes AI as national priority', 'media': 'The Korea Times', 'date': '2 months ago', 'datetime': datetime.datetime(2025, 11, 4, 19, 42, 14, 594356), 'desc': "President Lee Jae Myung on Tuesday called on the National Assembly to approve next year's budget, unveiling a record 728 trillion won ($525 billion...", 'link': 'https://www.koreatimes.co.kr/southkorea/politics/20251104/president-unveils-record-annual-budget-emphasizes-ai-as-national-priority&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBBAC&usg=AOvVaw2PwPBTjxBsV_xMNeY_Xu08', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'Federal government plans tax changes to enable $1 trillion boost to economy', 'media': 'BNN Bloomberg', 'date': '2 months ago', 'datetime': datetime.datetime(2025, 11, 4, 19, 42, 14, 602917), 'desc': "Prime Minister Mark Carney's government announced plans to enable $1 trillion in investments to drive productivity and boost the Canadian economy to enhance...", 'link': 'https://www.bnnbloomberg.ca/business/2025/11/04/federal-government-plans-tax-changes-to-enable-1-trillion-boost-to-economy/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIAhAC&usg=AOvVaw1JcO6iTEXe2vU6Mj8vRFdL', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': "Key targets in Greece's 2026 draft budget", 'media': 'Reuters', 'date': '2 months ago', 'datetime': datetime.datetime(2025, 11, 4, 19, 42, 14, 611457), 'desc': 'The Greek government submitted its 2026 draft budget plan to parliament on Monday, in which it forecasts economic growth of 2.4% in 2026 - outperforming...', 'link': 'https://www.reuters.com/business/key-targets-greeces-2026-draft-budget-2025-10-06/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBRAC&usg=AOvVaw3LbDIRxJO0x8SFE5a-cYzg', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}, {'title': 'Korea boosts R&D budget to record 35.3 trillion won for innovation and growth - CHOSUNBIZ', 'media': 'Chosunbiz', 'date': '4 months ago', 'datetime': datetime.datetime(2025, 9, 4, 19, 42, 14, 619989), 'desc': 'Minister Bae Kyung-hun of the Ministry of Science and ICT visits the Korea Research Institute of Standards and Science (KRISS) in Yuseong-gu, Daejeon,...', 'link': 'https://biz.chosun.com/en/en-science/2025/08/22/H4U3MOUSRNAFVH6EXGR7KFJQCE/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIARAC&usg=AOvVaw2y7lLm9MMFwuiF8hczDFF7', 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}]
    <<<<<<< 최종 답변 >>>>>>
    {'tools': {'messages': [ToolMessage(content='[{\'title\': \'해외 축구 라이브 의 비밀을 풀다: 전문가들이 공유하는 핵심 팁\', \'media\': \'www.termokonteiner.ru\', \'date\': \'2 weeks ago\', \'datetime\': datetime.datetime(2025, 12, 21, 19, 42, 14, 585515), \'desc\': \'해외축구 무료실시간tv중계. 추경 예산안 통과에 인사하는 다카이치 총리와 각료들 [AFP=연합뉴스 자료사진 제공] 일본 정부가 고물가 대응,방위력 강화 등을 위해...\', \'link\': \'https://www.termokonteiner.ru/%3FGameID%3D2025-12-20/6637.html&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBhAC&usg=AOvVaw1gfXGg1PjyCfIDdh0vrBht\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'President unveils record annual budget, emphasizes AI as national priority\', \'media\': \'The Korea Times\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 594356), \'desc\': "President Lee Jae Myung on Tuesday called on the National Assembly to approve next year\'s budget, unveiling a record 728 trillion won ($525 billion...", \'link\': \'https://www.koreatimes.co.kr/southkorea/politics/20251104/president-unveils-record-annual-budget-emphasizes-ai-as-national-priority&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBBAC&usg=AOvVaw2PwPBTjxBsV_xMNeY_Xu08\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Federal government plans tax changes to enable $1 trillion boost to economy\', \'media\': \'BNN Bloomberg\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 602917), \'desc\': "Prime Minister Mark Carney\'s government announced plans to enable $1 trillion in investments to drive productivity and boost the Canadian economy to enhance...", \'link\': \'https://www.bnnbloomberg.ca/business/2025/11/04/federal-government-plans-tax-changes-to-enable-1-trillion-boost-to-economy/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIAhAC&usg=AOvVaw1JcO6iTEXe2vU6Mj8vRFdL\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': "Key targets in Greece\'s 2026 draft budget", \'media\': \'Reuters\', \'date\': \'2 months ago\', \'datetime\': datetime.datetime(2025, 11, 4, 19, 42, 14, 611457), \'desc\': \'The Greek government submitted its 2026 draft budget plan to parliament on Monday, in which it forecasts economic growth of 2.4% in 2026 - outperforming...\', \'link\': \'https://www.reuters.com/business/key-targets-greeces-2026-draft-budget-2025-10-06/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIBRAC&usg=AOvVaw3LbDIRxJO0x8SFE5a-cYzg\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}, {\'title\': \'Korea boosts R&D budget to record 35.3 trillion won for innovation and growth - CHOSUNBIZ\', \'media\': \'Chosunbiz\', \'date\': \'4 months ago\', \'datetime\': datetime.datetime(2025, 9, 4, 19, 42, 14, 619989), \'desc\': \'Minister Bae Kyung-hun of the Ministry of Science and ICT visits the Korea Research Institute of Standards and Science (KRISS) in Yuseong-gu, Daejeon,...\', \'link\': \'https://biz.chosun.com/en/en-science/2025/08/22/H4U3MOUSRNAFVH6EXGR7KFJQCE/&ved=2ahUKEwiNh-LD2fGRAxVN3jQHHc9dBIkQxfQBegQIARAC&usg=AOvVaw2y7lLm9MMFwuiF8hczDFF7\', \'img\': \'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\'}]', name='search_news', id='74b69f8a-fcbf-4546-8e72-8ce36addac82', tool_call_id='737427d2-d62f-49cc-85e1-8ad2fecc4661')]}}
    <<<<<<< 최종 답변 >>>>>>
    [{'type': 'text', 'text': "최근 AI(인공지능) 투자와 관련된 주요 뉴스를 정리해 드립니다. 글로벌 시장에서는 인프라 확충과 에너지 확보가 핵심 키워드로 떠오르고 있으며, 국내에서도 정부의 강력한 의지와 기업들의 기술 도입이 활발합니다.\n\n### 1. 글로벌 빅테크 및 투자 동향\n*   **소프트뱅크의 공격적 행보:** 소프트뱅크가 최근 AI 분야에 **40억 달러(약 5조 원 이상)**를 투자하며 인프라 기업인 '디지털브리지'를 인수하는 등 AI 생태계 확장에 박차를 가하고 있습니다.\n*   **AI 인프라 및 에너지 투자 가속:** 단순히 소프트웨어 개발을 넘어, AI 데이터 센터 가동을 위한 **전력망과 에너지 인프라**가 주요 투자처로 부상했습니다. '이튼(Eaton)'이나 '콴타 서비스(Quanta Services)'와 같은 전력 설비 기업들이 AI 성장의 수혜주로 주목받고 있습니다.\n*   **반도체 패권 경쟁:** TSMC는 AI 수요에 대응하기 위해 **2nm(나노) 공정** 개발에 집중하며 장기적인 투자 가치를 증명하고 있습니다. 이는 AI 시장의 일시적인 변동성에도 흔들리지 않는 하드웨어 경쟁력을 확보하려는 움직임입니다.\n\n### 2. 국내 주요 뉴스 및 정책\n*   **정부의 역대급 R&D 예산 편성:** 정부는 2025~2026년 예산안에서 **AI를 국가 전략적 최우선 순위**로 두고, R&D 예산을 역대 최대 규모인 약 **35.3조 원**으로 편성했습니다. 'AI G3(글로벌 3대 강국)' 도약을 목표로 하고 있습니다.\n*   **산업별 AI 도입 확산:** 한국 제약사들이 신약 개발 기간을 단축하기 위해 **AI 플랫폼**을 적극적으로 도입하고 있으며, NHN과 같은 게임사들도 AI를 통한 사업 반등을 꾀하며 투자 가치가 상향 조정되고 있습니다.\n\n### 3. 향후 투자 전망 (2026년 핵심 키워드)\n*   **AI 에너지 백본(Backbone):** AI 데이터 센터의 폭증하는 전력 수요를 감당할 수 있는 에너지 솔루션 기업들이 2026년까지 높은 확신을 가진 투자 아이디어로 꼽힙니다.\n*   **온디바이스(On-device) AI:** 클라우드뿐만 아니라 스마트폰, PC 등 기기 자체에서 구동되는 AI 기술과 관련된 반도체 및 부품 기업들에 대한 관심이 지속될 것으로 보입니다.\n\n**요약하자면:** 현재 AI 투자는 '모델 개발'의 단계를 넘어, 이를 가능하게 하는 **'에너지, 인프라, 초미세 반도체'**라는 물리적 토대로 자금이 집중되는 추세입니다. 국내외 정책적 지원 또한 매우 강력한 상황입니다.", 'extras': {'signature': 'Er8ECrwEAXLI2nx5Yn3vUbvgyHlJKpsU2RIe4ta+pQ+dWF/6Y/86qIjfE7bp5XxFia+6j6kULrmKeq5AsbSJ+703yZE/uBHolhU0odu6WyQ2QI/jblIQDFjs0sZiS54FnwP6MBfAfm3uL34wGZO4b6mKP2FudhU2sIhG0zextIJuYPUpbaqmQVjzYJRYrd6zZhKawq7DuOqW+NPFpFT4TrrwLO/tfgESAVEF9vIUB5J/XJPrhh+04TsgLEvpbJCBcj+7Z9F4AGxgn0uy/e5Q+kbC/tuNjncDV7c83nlPFeue9dUgGce13pbQiRxr9u0K7o2neG+TwEl0SYKHRv+pbElLJdueNKoTUOKlCcmM1pX4vSdDKMdYg6MDpuGFQcUw0x86Lc1WnNZnoVjM5CXx98COCh4zMDNuyvVdNjx7R+625ANJneUbjMDoBzdEOz7eKX3w/xT9lFZ6P6nC3VuDiPr6EitLVd3AfBFDkMzHse56OEVRw+jJlvjPEpn61Xhahbx3b0/0uVLMJ0BVYBihCsKTIk/RWnYJB3q0WaW48KQCRgwBShzDFI42p4FOdpXflj0OSlqvxNlgXepOcKorULtR8N9SOfTQe7JMI2lbiVB6Hm2uW273RVMA+ByX97b9Yac2+fFr8u1+/wqoB/3CMOQhJ2066h3NXrJ62n3CNJ4ZN4juqhNjTnFx+WL7T97H/0q4N4UWA2bjZbP4HATWcPGwEQbOuiL5slEvT8AUVe3e5RaAjuqv0zXV3Ir7wv0Nioc='}}]
    <<<<<<< 최종 답변 >>>>>>
    ```

---

##### **7) 이전 대화내용을 기억하는 Agent**

* **`RunnableWithMessageHistory`** 를 사용 → **`AgentExecutor` 를 감싸줌**

* **[`RunnableWithMessageHistory`](https://wikidocs.net/254682)**

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 1. 메모리 저장소 생성 (In-memory)
memory = MemorySaver()

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}

In [ ]:
# session_id 를 기반으로 세션 기록을 가져오는 함수

def get_session_history(session_ids):

    # session_id 가 store에 없는 경우
    if session_ids not in store:  

        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()

    # 해당 세션 ID에 대한 세션 기록 반환
    return store[session_ids]  

<small>

* 교재 속 코드 `ERROR` 발생 

    ```python
    # 채팅 메시지 기록이 추가된 에이전트 생성하기
    agent_with_chat_history = RunnableWithMessageHistory(
        agent,
        get_session_history,        # 대화 session_id
        # 프롬프트의 질문이 입력되는 key: "input"
        input_messages_key="input",
        # 프롬프트의 메시지가 입력되는 key: "chat_history"
        history_messages_key="chat_history",
    )
    ```

* `LangGraph` 에이전트의 표준 메모리 방식인 `MemorySaver` *(Checkpointer)* 사용하는 코드로 수정하기

In [ ]:
# 2. 메모리 기능이 추가된 에이전트 다시 생성
# checkpointer를 인자로 전달하면 자동으로 대화 기록이 연동됩니다. (LangGraph 기능)
agent_with_chat_history = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_instruction,
    checkpointer=memory,
)

---

<small>

* 교재 속 코드

    ```python
    result3 = agent_with_chat_history.stream(
        {"messages": [HumanMessage(content="안녕? 내 이름은 앨리스야.")]},
        config={"configurable": {"session_id": "abc123"}},
        )

    for step in result3:
        # 중간 단계 출력 
        print(step)
    ```

    ↓

<small>

* `gemini-2.5-flash` 셀 출력 (`1.1s`)

    ```bash
    {'model': {'messages': [AIMessage(content='안녕하세요, 앨리스님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b88b9-5457-7d63-8247-4a68e93ab366-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 151, 'output_tokens': 13, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}})]}}
    ```

<small>

* `gemini-3-flash-preview` 셀 출력 (`2.4s`)

    ```bash
    {'model': {'messages': [AIMessage(content=[{'type': 'text', 'text': '반가워요, 앨리스! 만나서 정말 기뻐요. 😊\n\n저는 당신을 도와드리는 인공지능 비서예요. 오늘 하루는 어떻게 보내고 있나요? 혹시 제가 도와줄 일이 있거나, 같이 이야기 나누고 싶은 주제가 있다면 편하게 말씀해 주세요!', 'extras': {'signature': 'EpEFCo4FAXLI2nzKmVR+YCX9gt+9e1rB2aILlUpuzrM4XwJvVC9hmEBVlmZ6a25/HikpKw76T3AebsOyDT7PJ6niQztmSmqbWZaF3PZ1r3K9pYI3W+wvD/eQjE46hYlVMU3QJfwK18v/mx9LOgeJnmHAGdnasZMjutt9xsq+YSc9V+5zv0JfxDTyZ0W3YV7QjZpPkWFccRIhKrCFHXGZ5SAcigjl8M3h3Z/RERO3/iFe5fLGWCqaL2asWbcmHlCHjMOGkwEDF4VsHKMQOTFCm+CAZYLmd6tp3lbus28R+FdxVxbrCQVUI4jJhPAaTxYBTUGNBZcBfkqXWJCfCLDCKpcbKivUMCVBzehUwD+0oTadQTcT6S7T6NW7Xp5vUaQg1frg9Ie4OIS8X84+lzt53ySpyGFezTcMe2/rv8afZnvMWQqdbCfNRhI7AGUZyUGzvDwE1skvmwIsPhofzf2fY7WSI6ZFn3VkiU8gWlJn6Ays81ZFVhYEvqfq6VGXX4zB91mA6Z08TV7UHTTYSKgqbdh9GNV+VcaCXU8TZFKH5PFWBNNXufiu+hSm4trxZXFDhe3wUgaor/Mhus65cXIscshCecf7CDT40DWOB3uo/h9DkGdf3aTkXH9nTga1Sr5bSa5f3O9wxaLHOX/iZ4SsbB4FIsXx4ITivlhkfYdXEIxr4fizhTccHC28RNpxMGB7cTZzEDU+fr+p5j+4jkD8xUcOn8Olw/c+sRBWEdOPRH6hauyu2CydHlBxUu/Qfqg8ukr8rF8aPzs8+n3xys6xWbJa1a/ZLOopNNWJGSZXSzwR8t5CopEC3Se3bvDalDXlW+4WxU/64bRa4nky34Kx7g/AqqjKKNAeR74fXCJJ2r4N+qyw'}}], additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-3-flash-preview', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b88a2-babe-7e03-92ef-1012d6d6dfcd-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 167, 'output_tokens': 231, 'total_tokens': 398, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 167}})]}}
    ```

<small>

* or

    ```python
    # 질의에 대한 답변을 스트리밍으로 출력 요청
    result3 = agent_with_chat_history.stream(
        {"messages": [HumanMessage(content="안녕? 내 이름은 앨리스야.")]},
        config={"configurable": {"session_id": "abc123"}},
        )

    # 사용자 정의 파서 함수로 출력하기
    # 함수에 스트림 객체를 통째로 전달
    # 함수 안에서 for 루프를 돌면서 하나씩 꺼내 처리함
    parse_agent_stream(result3)
    ```

↓

<small>

* `gemini-2.5-flash` 셀 출력 (`1.2s`)

  * 🤖 AI 답변: 안녕하세요 앨리스님! 무엇을 도와드릴까요?

* *`gemini-3-flash-preview` 할당량 부족으로 실행 ❌*

<small>

* 교재 속 `session_id`로 실행 → **`ERROR`**

    ```python
    # 질의에 대한 답변을 스트리밍으로 출력 요청하기

    result3 = agent_with_chat_history.stream(
        {"messages": [HumanMessage(content="내 이름이 뭐라고?")]},
        # session_id 설정
        config={"configurable":{"session_id":"abc123"}},
        )

    # 출력 확인하기
    parse_agent_stream(result3)
    ```

↓

* 셀 출력 (`0.9s`)
  * 🤖 AI 답변: 저는 당신의 이름을 모릅니다.

---

* `session_id` ❌ → **`thread_id` ⭕️**

In [ ]:
# 3. 대화 실행 (thread_id 설정)

config = {"configurable": {"thread_id": "abc123"}}

result3 = agent_with_chat_history.stream(
    {"messages": [HumanMessage(content="안녕? 내 이름은 앨리스야.")]},
    config=config
    )
for step in result3:
    # 'model' 단계의 결과가 있을 때 텍스트만 출력
    if "model" in step:
        messages = step["model"]["messages"]
        for message in messages:
            if isinstance(message, AIMessage) and message.content:
                print(f"🤖 : {message.content}")

<small>

* `gemini-2.5-flash` 셀 출력 (`1.3s`)

  * 🤖 : 안녕하세요 앨리스님! 무엇을 도와드릴까요?

* *`gemini-3-flash-preview` 할당량 부족으로 실행 ❌*

In [ ]:
# 4. 기억력 테스트
result4 = agent_with_chat_history.stream(
    {"messages": [HumanMessage(content="내 이름이 뭐라고?")]},
    # 동일한 thread_id 사용
    config=config
    )

for step in result4:
    if "model" in step:
        messages = step["model"]["messages"]
        for message in messages:
            if isinstance(message, AIMessage) and message.content:
                print(f"🤖 : {message.content}")

<small>

* `gemini-2.5-flash` 셀 출력 (`1.1s`)
  * 🤖 : 앨리스님이시죠! 제가 맞게 기억하고 있네요.

* *`gemini-3-flash-preview` 할당량 부족으로 실행 ❌*

---

* 다른 질문: *질의에 대한 답변을 스트리밍으로 출력 요청하기*

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result5 = agent_with_chat_history.stream(
    {"messages": [HumanMessage(
        content="내 이메일 주소는 alice@alice.com 이야. 회사 이름은 앨리스 주식회사야.")]},
    # 동일한 thread_id 사용
    config=config
    )

# 출력 확인
for step in result5:
    if "model" in step:
        messages = step["model"]["messages"]
        for message in messages:
            if isinstance(message, AIMessage) and message.content:
                print(f"🤖 : {message.content}")

<small>

* `gemini-2.5-flash` 셀 출력 (`1.3s`)

  * 🤖 : 알겠습니다. 앨리스님의 이메일 주소는 alice@alice.com이고, 회사 이름은 앨리스 주식회사로 기억하겠습니다.

* *`gemini-3-flash-preview` 할당량 부족으로 실행 ❌* 

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result6 = agent_with_chat_history.stream(
    {"messages": [HumanMessage(
        content="Meta 관련 최신 뉴스 5개를 검색해서 제목을 나열해 이메일의 본문으로 작성해줘."
        "수신인에는 '셜리 상무님'을 적어주고, 발신인에는 내 인적정보를 적어줘."
        "정중한 어조로 작성하고, 메일의 시작과 끝에는 적절한 인사말과 맺음말을 적어줘.")]},
    # 동일한 thread_id 사용
    config=config
    )

# 출력 확인
for step in result6:
    if "model" in step:
        messages = step["model"]["messages"]
        for message in messages:
            if isinstance(message, AIMessage) and message.content:
                print(f"🤖 : {message.content}")

<small>

* `gemini-2.5-flash` 셀 출력 (`5.5s`)

    ```markdown
    🤖 : [{'type': 'text', 'text': "수신: 셜리 상무님\n\n안녕하십니까, 셜리 상무님.\n\n앨리스 주식회사 앨리스입니다.\n\nMeta 관련 최신 뉴스 5개를 정리하여 공유드립니다.\n\n1. Rooted Wealth Advisors Inc. Boosts Stake in Meta Platforms, Inc. $META\n2. HM Payson & Co. Acquires 1,126 Shares of Meta Platforms, Inc. $META\n3. Fort Washington Investment Advisors Inc. OH Sells 15,209 Shares of Meta Platforms, Inc. $META\n4. LeCun Exposes Meta's Ranking Cheating: Tian Yuandong Says Didn't Expect This Ending\n5. Copeland Capital Management LLC Sells 927 Shares of Meta Platforms, Inc. $META\n\n다른 궁금한 점이나 필요하신 정보가 있으시면 언제든지 편하게 말씀해주세요.\n\n감사합니다.\n\n앨리스 드림\n앨리스 (alice@alice.com)\n앨리스 주식회사", 'extras': {'signature': 'CowFAXLI2nz43oGyG+DO+DL1ijZ0uBtRik0it3BJNuZsFVGr0owJteGXwOHLzujBimC0pnrfx7qJgx52+XTOMGx3gT+9uW8rd9MuUIUXrJF34E0KKQuU7P0PJT9kFkeXSFAjh9Ml5IMzt6ljaDYBcvPuM0PMA5SJnfzVjSpcVZxo4PlFoQmFpQLovespfDFwLWLHUICfD1B93Ktibq21E97QkF9LULfZO6lvsRxAIfe1jfFKvOlCLs1hzyYIY1PEsrGg6UNyCXlYasqfsueL32uanMTTC6w/PN/Yr5dxBB7LirurLV1iXp/KzC76RdN8VbfwEw9K7isG2ReLDigcqhSml0djGQRUj6uU9RZMaqvSrRy6pLNux1ohOU0AFFuV1IOb9TtELDnN1O7c6O5AcKhgDEjKgRYhaTZzWWn1yQU8CnijKGEVuDP7Iy/OQ3uKtnQ4+P/VNpo1JWMlEGEyUZYLgi9zMK7ihp3OzMHlReSU4QTOOqR1cNOvmBigofmkFo8uCoqr/xeNiwUYSjSZWy4vFfZAPCIqFSFVnnyrr+JEX/vn/QtdjsOG8GdMe17FYVH0PKPT5nBbYLpvWogyKq+WvKRqFcCim34N1SzBCX/HFEU2lOfxrD9G19zsPSOM15iM34cRJDcNnxyUYPaQc67Oz38avF1WGYA3iLThU6p6a1nCYSMdtLc54Uk5hFqifi2sXpERSgSN/jHg/1aHWS06QUUaIgx6bhqfspod1ZgjD0fk5j5zctChPwHbB3RMIONI62NuxkEXY78EYjG/G9DBkq8+uidsHqLHIpr+C/sZ6etJaeYwc9EWFisP4Jhlgn6wC0Ru+88Qqk0yaREJW9H1anYgEN2TdaTGk5+v8w=='}}]
    ```

* *`gemini-3-flash-preview` 할당량 부족으로 실행 ❌* 

---

* next: ***`04. Claude, Gemini, Ollama, Together.ai 를 활용한 Agent`***

---